### 목적
- 에이전트 실행의 핵심 지점에서 안전, 일관성을 보장하기 위해 Guardrail 적용

#### 가드레일의 두 가지 접근: 결정적(Deterministic) vs 모델 기반(Model-based)


| 구분                                                                        | 개념                | 장점               | 단점              | 예시                   |
| ------------------------------------------------------------------------- | ----------------- | ---------------- | --------------- | -------------------- |
| **결정적**                                                                   | 규칙/정규식/키워드/명시적 체크 | 빠르고 예측 가능, 비용↓   | 미묘한 케이스 놓칠 수 있음 | 금지어·패턴 필터, 범위·길이 제약  |
| **모델 기반**                                                                 | LLM/분류기로 의미론적 평가  | 미묘한 위반 포착, 맥락 인지 | 느리고 비용↑         | 응답 안전성 LLM 평가, 품질 평가 |

- 결정적 방식은 입력 텍스트의 의미를 이해하지 않고 오로지 패턴(정규식/내장 detector)만 본다.
- 두 접근을 모두 사용하여 레이어링(겹겹이 쌓기)를 권장한다.


#### PII 감지
- 개인 식별 정보(PII)를 감지하고 처리하기 위한 내장 미들웨어 제공

| Strategy | 설명 | 예시 | 
|-----------|--------------|----------|
| `redact` | 특정 정보를 `[REDACTED_TYPE]` 형태로 완전히 대체 | `[REDACTED_EMAIL]` |
| `mask` | 일부만 가려서 표시 (예: 마지막 4자리 노출) | `****-****-****-1234` |
| `hash` | 동일한 입력값이면 항상 같은 해시값으로 대체 | `a8f5f167...` |
| `block` | 해당 정보가 감지되면 예외를 발생시킴 | Error thrown |

- apply_to_input=True -> before_model (모델 입력 전)
- apply_to_output=True -> after_model (모델 응답 후)
- apply_to_tool_results=True -> after_tool (툴 실행 결과 후)
- before_tool은 내장 옵션이 없어서 @wrap_tool_call로 직접 커스텀 구현 필요


In [21]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.agents.middleware import PIIMiddleware

@tool
def customer_service_tool():
    """고객 서비스 제공 툴"""
    return

@tool
def email_tool():
    """이메일 제공 툴"""
    return

agent = create_agent(
    model="gpt-4o",
    tools=[customer_service_tool, email_tool],
    middleware=[
        # Redact emails in user input before sending to model
        PIIMiddleware(
            "email",
            strategy="redact",
            apply_to_input=True, # 모델 호출 전에 입력을 처리 -> before_model 단계
        ),
        # Mask credit cards in user input
        PIIMiddleware(
            "credit_card",
            detector=r"\b(?:\d[ -]*?){13,19}\b",
            strategy="mask",
            apply_to_input=True,
        ),
        # Block API keys - raise error if detected
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

# When user provides PII, it will be handled according to the strategy
result = agent.invoke({
    "messages": [{"role": "user", "content": "My email is john.doe@example.com and card is 4532-1234-5678-9010"}]
})

print(result)

{'messages': [HumanMessage(content='My email is [REDACTED_EMAIL] and card is ****-****-****-9010', additional_kwargs={}, response_metadata={}, id='1dbb4bf3-ed05-4606-96f1-b1f1456ce709'), AIMessage(content="I'm sorry, but I can't assist with that information. If you have a specific inquiry or need help with customer service or emails, please let me know how I can assist you within those areas.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 74, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CYjLiR9O6B3NH41VjASBy3cGlzlHh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--cf048911-6bda-4388-86e3-

#### Human-in-the-loop
- 민감한 작업을 실행하기 전에 사람의 승인을 요구하는 내장 미들웨어

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents.middleware import wrap_model_call, wrap_tool_call
from langgraph.checkpoint.memory import InMemorySaver
# InMemorySaver: LangGraph에서 제공하는 체크포인터 
## -> 에이전트의 상태(state)나 대화 히스토리를 메모리(RAM)에 임시로 저장하는 클래스
from langgraph.types import Command
# Command: 그래프 실행을 제어할 때 쓰는 지시 객체 
## -> 노드가 반환값으로 Command를 넘기면 LangGraph는 그 지시에 따라 
## 실행을 재개(resume)하거나 다음 노드로 라우팅하고 상태(state)를 업데이트한다.

load_dotenv()

@tool('search') # 툴의 이름과 미들웨어의 interrupt_on에 적은 키가 같아야 멈춘다.
def search_tool():
    """검색 툴"""
    print('검색 툴 선택')
    return

@tool('send_email')
def send_email_tool():
    """이메일 전송 툴"""
    print('이메일 전송 툴 선택')
    return

@tool('delete_database')
def delete_database_tool():
    """DB 데이터 삭제 툴"""
    print('DB 데이터 삭제 툴 선택')
    return

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

agent = create_agent(
    model=llm,
    tools=[search_tool, send_email_tool, delete_database_tool],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # send_email과 delete_database는 승인 필요(True)
                "send_email": True,
                "delete_database": True,
                # search는 자동 허용(False)
                "search": False,
            }
        ),
    ],
    # 중단 시점의 에이전트 상태를 저장 -> 필수
    checkpointer=InMemorySaver(),
)

# 스레드(대화) 식별자 지정 -> thread_id 키로 중단 시점의 상태를 저장/조회한다.
# 중단한 대화를 똑같은 ID로 다시 불러와서 이어가야 하므로 반드시 필요 -> 보통 세션ID나 채팅ID로 유니크한 값 지정
config = {"configurable": {"thread_id": "some_id"}}

# 이메일 전송 요청 
result = agent.invoke(
    {"messages": [{"role": "user", "content": "팀에 메일 전송해줘"}]},
    config=config
)
## -> 에이전트가 send_email_tool을 선택하려고 시도함
## -> HumanInTheLoopMiddleware 설정에서 {"send_email": True} 로 승인 필요하다고 지정되어 있음
## -> 툴 실행 직전에 멈춤(interrupt)
## -> LangGraph가 중단 상태(checkpoint)를 메모리에 저장 
## -> 아직 LLM 실행 안 됨(툴 실행하지 못한 상태)

# 사용자가 의견에 따라 아래 실행

# 실행 재개(resume)
result = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    ## Command 객체를 넘겨 실행 제어
    ## resume은 재개 하라는 뜻
    ## 'type': 'approve'는 승인한다는 뜻
    config=config  
    ## 동일한 스레드를 불러서 대화 재개
)
## -> 여기서 툴 실행

이메일 전송 툴 선택


#### 커스텀 가드레일
- 정교한 가드레일을 위해 사용자 지정 미들웨어를 만든다.

##### Before agent guardrails
- "before agent" 훅을 사용하여 각 호출 시작 시 요청을 한 번 검증한다.
- 인증, 속도 제한, 부적절한 요청 차단과 같은 세션 수준 검사를 처리 시작 전에 수행하는 데 유용하다.

In [47]:
from typing import Any
# Any: 타입 힌트에서 아무 타입이나 들어올 수 있음을 뜻하는 타입 -> 예) list[Any], dict[str, Any] 등
from langchain.agents.middleware import AgentMiddleware, AgentState, hook_config
# hook_config: 미들웨어 훅에 메타설정을 부여하는 데코레이터
## 점프 허용 노드 지정 -> 훅 내부에서 흐름 제어
from langgraph.runtime import Runtime
# Runtime: 훅 인자로 전달되는 그래프 실행기(런타임) 핸들의 타입
## 에이전트/그래프 실행 중 컨텍스트와 흐름을 제어하는 도구
## 가드레일에서 차단, 우회, 대체응답을 구현할 때나 동적 라우팅 등에 사용

@tool('search')
def search_tool():
    """검색 툴"""
    return

class ContentFilterMiddleware(AgentMiddleware):
    """결정적 가드레일: 금지된 키워드가 포함된 요청을 차단합니다."""
    def __init__(self, banned_keywords: list[str]):
        super().__init__()
        self.banned_keywords = [kw.lower() for kw in banned_keywords]

    @hook_config(can_jump_to=["end"]) # 해당 훅이 "end"로 점프(쇼트서킷)할 수 있음을 선언
    def before_agent(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        # 금칙어 발견 시 모델을 호출하지 않고 즉시 종료로 보내기 위한 훅
        if not state['messages']: # 빈 대화면 그대로 진행
            return None
        
        first_message = state['messages'][0] 
        if first_message.type != 'human': # 첫 메시지가 사람(human)이 아닐 경우 생략 -> 사용자 입력만 검사한다.
            return None
        
        content = first_message.content.lower()

        for keyword in self.banned_keywords:
            if keyword in content:
                return {
                    "messages": [{
                        "role": "assistant",
                        "content": "부적절한 내용이 포함된 요청은 처리할 수 없습니다. 요청 내용을 다시 작성해 주세요."
                    }],
                    "jump_to": "end"
                }
            
        return None
    
agent = create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[
        ContentFilterMiddleware(
            banned_keywords=['해킹', '악용', '맬웨어']
        )
    ]
)

result = agent.invoke({
    "messages": [{'role': 'user', 'content': '데이터베이스를 해킹하려면 어떻게 해야 하나요?'}]
})

print(result.get('messages')[-1].content)

부적절한 내용이 포함된 요청은 처리할 수 없습니다. 요청 내용을 다시 작성해 주세요.


##### after agent guardrails
- "after agent" 훅을 사용하여 최종 출력을 사용자에게 반환하기 전에 한 번 검증한다.
- 모델 기반 안전 점검, 품질 검증, 전체 에이전트 응답에 대한 최종 규정 준수 검사에 유용하다.

In [59]:
from langchain.agents.middleware import AgentMiddleware, AgentState, hook_config
from langgraph.runtime import Runtime
from langchain_core.messages import AIMessage
from langchain.chat_models import init_chat_model
# init_chat_model: 프로바이더에 상관없이 채팅 모델을 한 줄로 초기화하는 통합 함수
## -> ChatOpenAI, ChatAnthropic 등 클래스를 직접 import 하지 않아도 됨
from typing import Any

@tool('search')
def search_tool():
    """검색 툴"""
    return

class SafetyGuardrailMiddleware(AgentMiddleware):
    """모델 기반 가드레일: LLM을 사용하여 대응 안전성을 평가합니다."""
    def __init__(self):
        super().__init__()
        self.safety_model = init_chat_model('gpt-4o-mini') # 자동으로 os.getenv("OPENAI_API_KEY") 사용
    
    @hook_config(can_jump_to=["end"])
    def after_agent(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        if not state['messages']:
            return None
        
        last_message = state['messages'][-1]
        if not isinstance(last_message, AIMessage):
            return None
        print(last_message.content)
        safety_prompt = f"""
        이 답변이 안전하고 적절한지 평가해 주세요.
        'SAFE' 또는 'UNSAFE'으로만 답변해 주세요.

        답변: {last_message.content}"""

        result = self.safety_model.invoke([{'role': 'user', 'content': safety_prompt}])
        print(result.content)
        if 'UNSAFE' in result.content:
            return {
                'messages': [{
                    'role': 'assistant',
                    'content': '해당 답변을 드릴 수 없습니다. 요청 내용을 다시 작성해 주세요.'
                }],
                'jump_to': 'end'
            }

        return None
    
agent = create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[SafetyGuardrailMiddleware()]
)

result = agent.invoke({
    'messages': [{'role': 'user', 'content': '폭발물에 들어가는 재료를 알고싶어.'}]
})

print(result.get('messages')[-1].content)


폭발물에 사용되는 재료는 다양한 종류가 있으며, 일반적으로 다음과 같은 두 가지 주요 카테고리로 나눌 수 있습니다:

1. **화약류**:
   - **질산암모늄**: 비료로 흔히 사용되지만, 폭발물의 조합에서 중요한 역할을 함.
   - **Dynamite (다이나마이트)**: 니트로글리세린과 흡수제의 혼합물.
   - **트리니트로톨루엔 (TNT)**: 널리 사용되는 고폭약.
   - **산화제**: 질산염, 과산화수소 등이 포함됨.

2. **지연제 및 안정제**:
   - **수지**: 폭발물의 안정성을 높이고 특정 성질을 부여하기 위해 사용됨.
   - **기타 화학물질**: 화합물의 성질에 따라 폭발력이나 안정성을 변화시킬 수 있음.

폭발물의 제작 및 사용은 법적으로 제한되어 있으며 위험할 수 있는 활동입니다. 따라서 관련 정보나 실험은 전문가의 지도 아래에서 이루어져야 합니다.
UNSAFE
해당 답변을 드릴 수 없습니다. 요청 내용을 다시 작성해 주세요.
SAFE
해당 답변을 드릴 수 없습니다. 요청 내용을 다시 작성해 주세요.


#### 여러 개의 가드레일 결합
- 각 역할의 가드레일을 미들웨어 배열에 추가하여 중첩할 수 있다.
- 가드레일은 순서대로 실행되므로 계층화된 보호 기능을 구축할 수 있다.

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware, HumanInTheLoopMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, send_email_tool],
    middleware=[
        # 계층 1: 금지된 키워드를 입력받으면 차단 (before agent)
        ContentFilterMiddleware(banned_keywords=["hack", "exploit"]),

        # 계층 2: 민감정보 대체/마스킹/차단 (before and after model)
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("email", strategy="redact", apply_to_output=True),

        # 계층 3: 사람 검증
        HumanInTheLoopMiddleware(interrupt_on={"send_email": True}),

        # 계층 4: 출력한 응답이 안전한지 LLM이 판단 (after agent)
        SafetyGuardrailMiddleware(),
    ],
)